### <font color=red>2018/11/2<br>调节了temp[item+'_ctr']中的细节，即将traindf中与testdf相同的item的_click,_count,_ctr赋值给testdf中，而不是直接使用testdf['label']=-1进行ctr计算<font>  
### 之前用了data.groupby()的成绩为0.6231，非常的差！过拟合现象非常严重！

In [1]:
import numpy as np
import pandas as pd
import warnings
warnings.filterwarnings('ignore')
import gc
gc.collect()

0

In [2]:
data=pd.read_csv('../data/data.csv')

In [3]:
data.shape

(2099998, 5)

In [4]:
data['query_len']=data['query'].apply(lambda x:len(x.replace('{','').replace('}','').split(',')))
# print(data[data['label']!=-1].groupby('query_len')['label'].mean())
# print('====================')
data['prefix_len']=data['prefix'].apply(lambda x:len(str(x)))
# print(data[data['label']!=-1].groupby('prefix_len')['label'].mean())
# print('====================')
data['tag_len']=data['tag'].apply(lambda x:len(str(x)))
# print(data[data['label']!=-1].groupby('tag_len')['label'].mean())
# print('====================')
data['title_len']=data['title'].apply(lambda x:len(str(x)))
# print(data[data['label']!=-1].groupby('title_len')['label'].mean())
# print('====================')
# print(data.shape)
# print('====================')
# print(data.columns)
# print('====================')
# print(data.describe())
# print('====================')
# print(data.info())

In [5]:
gc.collect()

28

In [6]:
from sklearn.preprocessing import LabelEncoder
le=LabelEncoder()
lr_col=['prefix','title','tag','query']
for feature in lr_col:
    data[feature]=data[feature].astype(str)

data['prefix']=le.fit_transform(data['prefix'])
data['title']=le.fit_transform(data['title'])
data['query']=le.fit_transform(data['query'])
data['tag']=le.fit_transform(data['tag'])

In [7]:
traindf=data[data['label']!=-1] #有一个标签是音乐,可以直接删除这个样本
testdf=data[data['label']==-1]
print(traindf.shape)
print(testdf.shape)

(2049998, 9)
(50000, 9)


In [8]:
#单项历史点击率特征构造
col=['prefix','query','title','tag','query_len','prefix_len','tag_len','title_len']
for item in col:
    temp=traindf.groupby(item,as_index=False)['label'].agg({item+'_click':'sum',item+'_count':'count'})
    temp[item+'_ctr']=(temp[item+'_click']/temp[item+'_count'])
    traindf = pd.merge(traindf, temp, on=item, how='left')
    testdf=pd.merge(testdf,temp,on=item,how='left')
    #testdf中有很多是traindf中没有的，所以导致testdf.merger的时候就会出现很多NaN

In [9]:
#交叉特征历史点击率特征构造
cols=['prefix','query','title','tag','query_len','title_len']
for i in range(len(cols)):
    for j in range(i+1,len(cols)):
        item_g=[cols[i],cols[j]]
        temp=traindf.groupby(item_g,as_index=False)['label'].agg({'_'.join(item_g)+'_click':'sum','_'.join(item_g)+'_count':'count'})
        temp['_'.join(item_g)+'_ctr']=temp['_'.join(item_g)+'_click']/(temp['_'.join(item_g)+'_count']+3)
        traindf=pd.merge(traindf,temp,on=item_g,how='left')
        testdf=pd.merge(testdf,temp,on=item_g,how='left')

In [10]:
print(traindf.shape,testdf.shape)

(2049998, 78) (50000, 78)


In [11]:
print(pd.isnull(traindf).sum())
print("================")
print(pd.isnull(testdf).sum())

prefix                       0
query                        0
title                        0
tag                          0
label                        0
query_len                    0
prefix_len                   0
tag_len                      0
title_len                    0
prefix_click                 0
prefix_count                 0
prefix_ctr                   0
query_click                  0
query_count                  0
query_ctr                    0
title_click                  0
title_count                  0
title_ctr                    0
tag_click                    0
tag_count                    0
tag_ctr                      0
query_len_click              0
query_len_count              0
query_len_ctr                0
prefix_len_click             0
prefix_len_count             0
prefix_len_ctr               0
tag_len_click                0
tag_len_count                0
tag_len_ctr                  0
                            ..
query_title_click            0
query_ti

# ===========Train===============

In [11]:
# traindf['id']=0
# for i in range(traindf.shape[0]):
#     traindf['id'][i]=i

# nid=2000000
# testdf['id']=0
# for i in range(testdf.shape[0]):
#     testdf['id'][i]=i+

In [12]:
# traindf.to_csv('../data/final_train2.csv',index=False)
testddf=testdf.drop(['label'],axis=1)
# testddf.to_csv('../data/final_test2.csv',index=False)

In [13]:
y=traindf.pop('label').values
X=traindf.as_matrix()
testX=testddf.as_matrix()

In [14]:
from lightgbm import LGBMClassifier
# from sklearn.linear_model import LogisticRegression
# from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import f1_score

In [15]:
sub=pd.DataFrame()

In [17]:
# lr = LogisticRegression(penalty='l2', dual=False, tol=0.0001, C=0.05, random_state=1,
#                             solver='liblinear', max_iter=100,verbose=1)

In [18]:
# def lr_fit(lr):
#     fs=0 #f1_score
#     mm=MinMaxScaler()
#     X=mm.fit_transform(X)
#     testX=mm.transform(testX)
#     for i,(train_index,test_index) in enumerate(skf.split(X,y)):
#         print('Fold:%s'%i)
#         lr.fit(X[train_index],y[trin_index])
#         y_pred=lr.predict(X[test_index])
        
#         fscore=f1_score(y[test_index],y_pred)
#         fs+=fscore
#         print("f1_score为：%.5f"%fscore)
        
#         X_pred=lr.predict_proba(X[train_index])  #10折后得到一个完整的traindf_feature
#         t_pred=lr.predict_proba(testX)  #10折后得到10个完整的testdf_feature,再对其取平均     

In [20]:
logloss=[]
losstrain=0
lossval=0
fs=0

N=10
#基本kfold和StratifiedKfold基本相同，唯一不同的是skf可以保证在训练集、测试集中的采样的类别比例相同
skf=StratifiedKFold(n_splits=N,random_state=1,shuffle=True)

clf= LGBMClassifier(boosting_type='gbdt', num_leaves=60, max_depth=-1, 
                         learning_rate=0.05, n_estimators=2000,objective='binary', 
                         min_split_gain=0,min_child_weight=5, min_child_samples=10, 
                         subsample=0.8, subsample_freq=1,colsample_bytree=0.8, 
                         reg_alpha=3, reg_lambda=5, random_state=1, n_jobs=-1)

In [24]:
for i,(train_index,test_index) in enumerate(skf.split(X,y)):
    print("Fold=%s"%(i))
    model=clf.fit(X[train_index],y[train_index],
                 eval_names=['train','valid'],
                 eval_metric='logloss',
                  eval_set=[(X[train_index],y[train_index]),(X[test_index],y[test_index])],
                 early_stopping_rounds=100,
                  verbose=50,)
    
    trainss=model.best_score_['train']['binary_logloss']
    valss=model.best_score_['valid']['binary_logloss']
    losstrain+=trainss
    lossval+=valss
    print("best logloss on train_set:",trainss)
    print("best logloss on test_set:",valss)
    
    val_proba=model.predict_proba(X[test_index],num_iteration=model.best_iteration_)[:,1]
    val_pred=np.where(val_proba>0.5,1,0)

    fval=f1_score(y[test_index],val_pred)
    print("f1_score on test_set:",fval)
    fs+=fval
    
    test_proba=model.predict_proba(testX,num_iteration=model.best_iteration_)[:,1]
    sub['ans_%s'%str(i)]=test_proba
print("train_set上最好的logloss成绩的平均值为：%.5f"%(losstrain/N))
print("val_set上最好的logloss成绩的平均值为：%.5f"%(lossval/N))
print("val_set上f1_score成绩的平均值为：%.5f"%(fs/N))

Fold=0
Training until validation scores don't improve for 100 rounds.
[50]	train's binary_logloss: 0.336972	valid's binary_logloss: 0.337136
[100]	train's binary_logloss: 0.32292	valid's binary_logloss: 0.323389
[150]	train's binary_logloss: 0.321464	valid's binary_logloss: 0.322258
[200]	train's binary_logloss: 0.321071	valid's binary_logloss: 0.322247
[250]	train's binary_logloss: 0.320882	valid's binary_logloss: 0.322456
Early stopping, best iteration is:
[178]	train's binary_logloss: 0.32121	valid's binary_logloss: 0.322213
best logloss on train_set: 0.321209729425755
best logloss on test_set: 0.32221322634427024
f1_score on test_set: 0.7940166549505219
Fold=1
Training until validation scores don't improve for 100 rounds.
[50]	train's binary_logloss: 0.337043	valid's binary_logloss: 0.336534
[100]	train's binary_logloss: 0.323041	valid's binary_logloss: 0.322684
[150]	train's binary_logloss: 0.32156	valid's binary_logloss: 0.321462
[200]	train's binary_logloss: 0.321189	valid's bin

In [34]:
sub['label']=0
for i in range(N):
    sub['label']+=sub['ans_%s'%str(i)]
sub['label']=sub['label']/N
sub['label']=sub['label'].apply(lambda x:np.where(x>0.40,1,0))

In [35]:
sub.groupby('label')['label'].count()

label
0    32911
1    17089
Name: label, dtype: int64

In [36]:
print(17089/50000)

0.34178


In [41]:
sub['label'].to_csv('../data/BaselineV1_2.csv',header=False,index=False)

<font color=red>2018/11/1<br>过拟合现象之所以严重，与XXY版本对比，可以发现，很大的原因是是因为feature之间相互作用而导致的！

## <font color=blue>2018/11/3<br>1.主要将datadf.groupby(item)['label].agg()修改为traindf.groupby(item,index=False)['label'].agg({sum,count})；<br><br>2.在最后将点击与否的界限设置为0.40；<br><br>3.得到的成绩为0.7128，排名429/2979<font>